## Integrated FM Control

In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem import utils
from dataclasses import dataclass, field

microscope, settings = utils.setup_session()

In [ ]:
# Controls
# Light Source 
    # Power
    # Exposure Time
# Filter Wheel
    # Excitation Wavelength
    # Emission Wavelength
# Objective Lens
    # Magnification
    # Position
    # Move Relative
    # Move Absolute
# Camera
    # Binning
    # Gain
    # Offset

# Tasks
# Acquire Image
# Acquire Z-Stack
# Acquire Overview
# Move to Position
# Save OME Metadata
# Load OME Metadata

# TODO:
# - tests
# 


In [ ]:
from typing import Optional
from abc import ABC, abstractmethod
import numpy as np
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
from pprint import pprint

from fibsem.fm.microscope import FluorescenceMicroscope, Camera, LightSource, ObjectiveLens, FilterSet
from fibsem.fm.structures import ChannelSettings, ZParameters
from fibsem.fm.acquisition import acquire_channels, acquire_z_stack, run_auto_focus

# create microscope 
fm = FluorescenceMicroscope()
objective = ObjectiveLens(parent=fm)
filter_set = FilterSet(parent=fm)
camera = Camera(parent=fm)
light_source = LightSource(parent=fm)
fm.objective = objective
fm.filter_sets = [filter_set]
fm.camera = camera
fm.light_source = light_source
# OME-standard -> Static instrument configuration vs dynamic image acquisition settings

zparams = ZParameters(zmin=-10e-6, zmax=10e-6, zstep=1e-6)

ch1 = ChannelSettings(
    name="Channel-01", 
    excitation_wavelength=488.0, 
    emission_wavelength=520.0, 
    power=100.0, 
    exposure_time=200.0, 
    binning=1,
)
ch2 = ChannelSettings(
    name="Channel-02", 
    excitation_wavelength=561.0, 
    emission_wavelength=None, 
    power=80.0, 
    exposure_time=150.0,
    binning=2,
)



In [ ]:
# acquire a single image
image = fm.acquire_image(ch1)
print(image)
print(f"Acquired image with shape: {image.data.shape} and dtype: {image.data.dtype}")
pprint(image.metadata)
plt.imshow(image.data, cmap='gray')
plt.title(f"Acquired Image - {ch1.name}")
plt.colorbar()
plt.show()



In [ ]:
# Acquire images for multiple channels
images = acquire_channels(fm, [ch1, ch2])
for idx, image in enumerate(images):
    print(f"Image {idx+1} Metadata:")
    md = image.metadata
    pprint(md)
    print(f"Image {idx+1} Shape: {image.data.shape}, Dtype: {image.data.dtype}")
    # Optionally display the image
    plt.imshow(image.data, cmap='gray')
    plt.title(f"Acquired Image - {md['objective']['magnification']}x")
    plt.colorbar()
    plt.show()



In [ ]:
# Acquire Z-Stack
image = acquire_z_stack(fm, ch1, zparams)
print(image.data.shape)
md = image.metadata

img: np.ndarray
for idx, img in enumerate(image.data):
    print(f"Z-Stack Image {idx+1} Metadata:")
    # pprint(md[idx])
    print(f"Z-Stack Image {idx+1} Shape: {img.shape}, Dtype: {img.dtype}")
    # Optionally display the image
    plt.imshow(img, cmap='gray')
    plt.title(f"Z-Stack Image - Z Position: {md['objective']['position']:.2e} microns")
    plt.colorbar()
    plt.show()


In [ ]:
# Run autofocus
fm.objective.move_absolute(0.0)  # Reset objective position to 0
best_focus = run_auto_focus(fm, ch1)
print(f"Best focus position: {best_focus:.2e} microns")